In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import os

E:\KNOWLEDGES\Computer_Science\ANACONDA\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\KNOWLEDGES\Computer_Science\ANACONDA\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\KNOWLEDGES\Computer_Science\ANACONDA\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\KNOWLEDGES\Computer_Science\ANACONDA\lib\site-packages\ten

In [3]:
## set parameters

ROOT = 'C:/Users/BuiQuocBao/OneDrive/Desktop/GRU4REC/processed/'
DATA_TYPE = 'sample'
PATH_TO_TRAIN = ROOT + 'rsc15_train_{}.txt'.format(DATA_TYPE)
PATH_TO_TEST = ROOT + 'rsc15_test_{}.txt'.format(DATA_TYPE)
checkpoint_dir = './checkpoint'
if not os.path.exists(checkpoint_dir): os.mkdir(checkpoint_dir)
        
layers = 1
rnn_size = 100
batch_size = 50
drop_keep_prob = 0.7

n_epochs = 3
learning_rate = 0.001
decay = 0.96
decay_steps = 1e4
grad_cap = 0
print_step = 1e3

In [4]:
## load data
data = pd.read_csv(PATH_TO_TRAIN, sep='\t', dtype={'ItemId': np.int64})
valid = pd.read_csv(PATH_TO_TEST, sep='\t', dtype={'ItemId': np.int64})

In [5]:
## add item index 
##
itemids = data['ItemId'].unique()
n_items = len(itemids)
itemidmap = pd.Series(data=np.arange(n_items), index=itemids).to_dict()
%time data['ItemIdx'] = data['ItemId'].map(lambda x: itemidmap[x])
data[:10]

Wall time: 2.07 s


,SessionId,ItemId,timestamp,ItemIdx
0,6,214701242,1.396804e+09,0
1,6,214826623,1.396804e+09,1
2,21,214838503,1.396861e+09,2
3,21,214838503,1.396861e+09,2
4,21,214838503,1.396861e+09,2
5,21,214838503,1.396862e+09,2
6,21,214838503,1.396862e+09,2
7,21,214548744,1.396862e+09,3
8,36,214551594,1.396814e+09,4
9,36,214586970,1.396814e+09,5


In [6]:
## offset sessions
## EX : SessionId 6 : start 0
##      SessionId 21 : start 2
##      SessionId 36 : start 8
offset_sessions = np.zeros(data['SessionId'].nunique()+1, dtype=np.int32)
offset_sessions[1:] = data.groupby('SessionId').size().cumsum()
offset_sessions[:10]

array([ 0,  2,  8, 10, 15, 19, 24, 27, 32, 34])

In [7]:
## PREPARE MODEL

In [8]:
## placeholder & learning rate
## tf.placeholder() : Allocated storage for data (such as for image pixel data during a feed)
X = tf.placeholder(tf.int32, [batch_size], name='input')
Y = tf.placeholder(tf.int32, [batch_size], name='output')
## for _ in range() : Ignore the index
States = [tf.placeholder(tf.float32, [batch_size, rnn_size], name='rnn_state') for _ in range(layers)]
## tf.Variable you have to provide an initial value when you declare it. 
## tf.placeholder you don't have to provide an initial value 
global_step = tf.Variable(0, name='global_step', trainable=False)
## When training a model, it is often recommended to lower the learning rate as the training progresses. 
## This function applies an exponential decay function to a provided initial learning rate. 
## It requires a global_step value to compute the decayed learning rate. 
## You can just pass a TensorFlow variable that you increment at each training step.
lr = tf.maximum(1e-5,tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay, staircase=True))

In [9]:
## gru weigths
##
with tf.variable_scope('gru_layer', reuse=tf.AUTO_REUSE):
    #sigma = sigma if sigma != 0 else np.sqrt(6.0 / (n_items + rnn_size))
    #initializer = tf.random_uniform_initializer(minval=-sigma, maxval=sigma)
    initializer = tf.glorot_uniform_initializer()
    # get_variable(name, shape, initializer) : Khoi tao bien
    # initializer : Initializer for the variable if one is created. Can either be an initializer object or a Tensor. 
    #               If it's a Tensor, its shape must be known unless validate_shape is False.
    embedding = tf.get_variable('embedding', [n_items, rnn_size], initializer=initializer)
    softmax_W = tf.get_variable('softmax_w', [n_items, rnn_size], initializer=initializer)
    softmax_b = tf.get_variable('softmax_b', [n_items], initializer=tf.zeros_initializer())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
## gru_cell
##
with tf.variable_scope('gru_cell', reuse=tf.AUTO_REUSE):
    # rnn_size : int, The number of units in the GRU cell.
    cell = tf.nn.rnn_cell.GRUCell(rnn_size, activation=tf.nn.tanh)
    # DropoutWrapper(cell, output_keep_prob)
    # cell : an RNNCell, a projection to output_size is added to it.
    # output_keep_prob : unit Tensor or float between 0 and 1, output keep probability; 
    #                    if it is constant and 1, no output dropout will be added.
    drop_cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=drop_keep_prob)
    # MultiRNNCell(cells, state_is_tuple=True) : RNN cell composed sequentially of multiple simple cells.
    # cells : list of RNNCells that will be composed in this order.
    stacked_cell = tf.nn.rnn_cell.MultiRNNCell([drop_cell] * layers)

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [11]:
## feedforward gur_cell
## EX : seesion 1 : item sequence 5, 7, 9
## outputs of 7 is final_state

inputs = tf.nn.embedding_lookup(embedding, X)
output, state_ = stacked_cell(inputs, tuple(States))
final_state = state_

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
## for training
sampled_W = tf.nn.embedding_lookup(softmax_W, Y)
sampled_b = tf.nn.embedding_lookup(softmax_b, Y)
logits = tf.matmul(output, sampled_W, transpose_b=True) + sampled_b

In [13]:
## cross-entropy loss
yhat = tf.nn.softmax(logits)
cost = tf.reduce_mean(-tf.log(tf.diag_part(yhat)+1e-24))

In [14]:
## for prediction
logits_all = tf.matmul(output, softmax_W, transpose_b=True) + softmax_b
yhat_all = tf.nn.softmax(logits_all)

In [15]:
## optimize
## Adam optimizer
optimizer = tf.train.AdamOptimizer(lr)
tvars = tf.trainable_variables()
gvs = optimizer.compute_gradients(cost, tvars)
if grad_cap > 0:
    capped_gvs = [(tf.clip_by_norm(grad, grad_cap), var) for grad, var in gvs]
else:
    capped_gvs = gvs 
train_op = optimizer.apply_gradients(capped_gvs, global_step=global_step)

In [16]:
## TRAINING

In [17]:
## session start
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())

In [18]:
## UNDERSTANDING DATA FEED

In [28]:
## data row index of start of session id
iters = np.arange(batch_size)
maxiter = iters.max()
print(iters[:5], "...", iters[-5:])
print(maxiter)
start = offset_sessions[iters]
end = offset_sessions[iters+1]
print(start[:10])
print(end[:10])

[0 1 2 3 4] ... [45 46 47 48 49]
49
[ 0  2  8 10 15 19 24 27 32 34]
[ 2  8 10 15 19 24 27 32 34 36]


In [29]:
## start item index of each session
minlen = (end-start).min()
print(data[:20])
out_idx = data.ItemIdx.values[start]
print(minlen)
print(out_idx[:10])

    SessionId     ItemId     timestamp  ItemIdx
0           6  214701242  1.396804e+09        0
1           6  214826623  1.396804e+09        1
2          21  214838503  1.396861e+09        2
3          21  214838503  1.396861e+09        2
4          21  214838503  1.396861e+09        2
5          21  214838503  1.396862e+09        2
6          21  214838503  1.396862e+09        2
7          21  214548744  1.396862e+09        3
8          36  214551594  1.396814e+09        4
9          36  214586970  1.396814e+09        5
10         41  214821277  1.396773e+09        6
11         41  214821277  1.396773e+09        6
12         41  214821277  1.396773e+09        6
13         41  214821277  1.396773e+09        6
14         41  214821277  1.396773e+09        6
15         53  214544355  1.396449e+09        7
16         53  214601212  1.396449e+09        8
17         53  214601212  1.396449e+09        8
18         53  214601212  1.396449e+09        8
19         56  214832557  1.396543e+09  

In [30]:
## in_idx : start item index of each session
## out_idx : end item index of each session
i = 0
in_idx = out_idx
out_idx = data.ItemIdx.values[start+i+1]
print(in_idx[:10])
print(out_idx[:10])

[ 0  2  4  6  7  9 13 16 18 20]
[ 1  2  5  6  8 10 14 16 19 21]


In [32]:
start = start+minlen-1

mask = np.arange(len(iters))[(end-start)<=1]
print(end[:10])
print(start[:10])
print(mask[:10])

[ 2  8 10 15 19 24 27 32 34 36]
[ 1  3  9 11 16 20 25 28 33 35]
[ 0  2  8  9 10 14 17 25 27 28]


In [33]:
print(len(mask))

22


In [34]:
iters[0] = 50
iters[2] = 51
print(iters[:10])
###
start[0] = offset_sessions[50]
end[0] = offset_sessions[50+1]
start[2] = offset_sessions[51]
end[2] = offset_sessions[51+1]
print(start[:10])
print(end[:10])

[50  1 51  3  4  5  6  7  8  9]
[186   3 188  11  16  20  25  28  33  35]
[188   8 191  15  19  24  27  32  34  36]


In [35]:
for idx in mask:
    maxiter += 1
    if maxiter >= len(offset_sessions)-1:
        finished = True
        break
    iters[idx] = maxiter
    start[idx] = offset_sessions[maxiter]
    end[idx] = offset_sessions[maxiter+1]
print(start[:10])
print(end[:10])

[186   3 188  11  16  20  25  28 191 195]
[188   8 191  15  19  24  27  32 195 198]


In [36]:
finished = False
endpoint_count = 0
while not finished:
    minlen = (end-start).min()
    out_idx = data.ItemIdx.values[start]
    for i in range(minlen-1):
        in_idx = out_idx
        out_idx = data.ItemIdx.values[start+i+1]
        endpoint_count += len(out_idx)
    
    start = start+minlen-1
    mask = np.arange(len(iters))[(end-start)<=1]

    for idx in mask:
        maxiter += 1
        if maxiter >= len(offset_sessions)-1:
            finished = True
            break
        iters[idx] = maxiter
        start[idx] = offset_sessions[maxiter]
        end[idx] = offset_sessions[maxiter+1]
        
print(max(start))
data[-5:]

3140308


,SessionId,ItemId,timestamp,ItemIdx
3140305,11562131,214854542,1.411823e+09,21584
3140306,11562151,214536296,1.411769e+09,3483
3140307,11562151,214536296,1.411769e+09,3483
3140308,11562157,214580372,1.411648e+09,1881
3140309,11562157,214516012,1.411648e+09,1880


In [37]:
print(endpoint_count)
print(sum(data.groupby('SessionId').size() - 1))

2348300
2348603


In [38]:
## TRAINING

In [ ]:
tic = time.time()
for epoch in range(n_epochs):
    epoch_cost = []
    state = [np.zeros([batch_size, rnn_size], dtype=np.float32) for _ in range(layers)]
    iters = np.arange(batch_size)
    maxiter = iters.max()
    
    start = offset_sessions[iters]
    end = offset_sessions[iters+1]
    
    finished = False
    while not finished:
        minlen = (end-start).min()
        out_idx = data.ItemIdx.values[start]
        for i in range(minlen-1):
            in_idx = out_idx
            out_idx = data.ItemIdx.values[start+i+1]
            # prepare inputs, targeted outputs and hidden states
            fetches = [cost, final_state, global_step, lr, train_op]
            feed_dict = {X: in_idx, Y: out_idx}
            for j in range(layers): 
                feed_dict[States[j]] = state[j]
            
            cost_, state, step, lr_, _ = sess.run(fetches, feed_dict)
            epoch_cost.append(cost_)
            
            if step == 1 or step % print_step == 0:
                avgc = np.mean(epoch_cost)
                print('Epoch {}\tStep {}\tlr: {:.5f}\tloss: {:.4f}\tElapsed: {:.1f}'.
                      format(epoch, step, lr_, avgc, time.time()-tic))

        start = start+minlen-1
        mask = np.arange(len(iters))[(end-start)<=1]
        for idx in mask:
            maxiter += 1
            if maxiter >= len(offset_sessions)-1:
                finished = True
                break
            iters[idx] = maxiter
            start[idx] = offset_sessions[maxiter]
            end[idx] = offset_sessions[maxiter+1]
        if len(mask):
            for i in range(layers):
                state[i][mask] = 0
        
    avgc = np.mean(epoch_cost)
    if np.isnan(avgc):
        print('Epoch {}: Nan error!'.format(epoch, avgc))
        break
    saver.save(sess, '{}/gru-model'.format(checkpoint_dir), global_step=epoch)
print("1 epoch elapsed time:", time.time() - tic)

Epoch 0	Step 1	lr: 0.00100	loss: 3.9120	Elapsed: 0.2
Epoch 0	Step 1000	lr: 0.00100	loss: 3.3758	Elapsed: 44.8
Epoch 0	Step 2000	lr: 0.00100	loss: 3.0282	Elapsed: 96.7
Epoch 0	Step 3000	lr: 0.00100	loss: 2.8469	Elapsed: 146.9
Epoch 0	Step 4000	lr: 0.00100	loss: 2.6973	Elapsed: 240.8
Epoch 0	Step 5000	lr: 0.00100	loss: 2.6036	Elapsed: 336.8
Epoch 0	Step 6000	lr: 0.00100	loss: 2.5199	Elapsed: 485.5
Epoch 0	Step 7000	lr: 0.00100	loss: 2.4696	Elapsed: 736.9
Epoch 0	Step 8000	lr: 0.00100	loss: 2.4117	Elapsed: 994.3
Epoch 0	Step 9000	lr: 0.00100	loss: 2.3730	Elapsed: 1246.8
Epoch 0	Step 10000	lr: 0.00100	loss: 2.3305	Elapsed: 1533.0
Epoch 0	Step 11000	lr: 0.00096	loss: 2.3054	Elapsed: 1813.5
Epoch 0	Step 12000	lr: 0.00096	loss: 2.2729	Elapsed: 2094.3
Epoch 0	Step 13000	lr: 0.00096	loss: 2.2506	Elapsed: 2375.4
Epoch 0	Step 14000	lr: 0.00096	loss: 2.2250	Elapsed: 2654.7
Epoch 0	Step 15000	lr: 0.00096	loss: 2.2004	Elapsed: 2942.8
Epoch 0	Step 16000	lr: 0.00096	loss: 2.1742	Elapsed: 3137.4
Epoch 

In [ ]:
## parameters
cut_off = 20     # @20
batch_size = 50

In [ ]:
## session restore
##
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.latest_checkpoint(checkpoint_dir)
saver.restore(sess, ckpt)

In [ ]:
## valdation data set
valid['ItemIdx'] = valid['ItemId'].map(lambda x: itemidmap[x])
valid[:5]

In [ ]:
offset_sessions = np.zeros(valid['SessionId'].nunique()+1, dtype=np.int32)
offset_sessions[1:] = valid.groupby('SessionId').size().cumsum()
offset_sessions[:5]

In [ ]:
## init prediction
##
if len(offset_sessions) - 1 < batch_size:
    batch_size = len(offset_sessions) - 1
### training step
iters = np.arange(batch_size).astype(np.int32)
maxiter = iters.max()
start = offset_sessions[iters]
end = offset_sessions[iters+1]
in_idx = np.zeros(batch_size, dtype=np.int32)
predict_state = [np.zeros([batch_size, rnn_size], dtype=np.float32) for _ in range(layers)]

In [ ]:
## prediction & evaluation
## data feeding 
evalutation_point_count = 0
mrr, recall = 0.0, 0.0
tic = time.time()
while True:
    valid_mask = iters >= 0
    if valid_mask.sum() == 0:
        print("break at endpoint", evalutation_point_count)
        break
        
    start_valid = start[valid_mask]
    minlen = (end[valid_mask]-start_valid).min()
    in_idx[valid_mask] = valid.ItemIdx.values[start_valid]
    
    for i in range(minlen-1):
        out_idx = valid.ItemIdx.values[start_valid+i+1]
        ## --- prediction --- ##
        fetches = [yhat_all, final_state]
        feed_dict = {X: in_idx}
        for j in range(layers): 
            feed_dict[States[j]] = predict_state[j]
        preds, predict_state = sess.run(fetches, feed_dict)
        preds = pd.DataFrame(data=np.asarray(preds).T)
        preds.fillna(0, inplace=True) ### preds shape: (item_size, batch_size)
        
        ## --- evaluation --- ##
        in_idx[valid_mask] = out_idx
        
        ranks = (preds.values.T[valid_mask].T > 
                 np.diag(preds.loc[in_idx].values)[valid_mask]).sum(axis=0) + 1
        
        rank_ok = ranks < cut_off
        recall += rank_ok.sum()
        mrr += (1.0 / ranks[rank_ok]).sum()
        evalutation_point_count += len(ranks)
        
    start = start+minlen-1
    mask = np.arange(len(iters))[(valid_mask) & (end-start<=1)]
    
    for idx in mask:
        maxiter += 1
        ## 
        if maxiter >= len(offset_sessions)-1:
            iters[idx] = -1
        else:
            iters[idx] = maxiter
            start[idx] = offset_sessions[maxiter]
            end[idx] = offset_sessions[maxiter+1]
            
    if len(mask):
        for i in range(layers):
            predict_state[i][mask] = 0

### 
recall = recall/evalutation_point_count
mrr = mrr/evalutation_point_count
print("recall: ", recall, "mrr:", mrr, "elapsed time:", time.time()-tic)